<a href="https://colab.research.google.com/github/ProjetoDados/Esquadr-o-Vendas/blob/main/Envio_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# 📬 Script de envio de e-mail + PDF mais recente do Drive

import smtplib
import os
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# 🔒 Dados do Gmail
email_remetente = "projetodadosnb@gmail.com"
senha_app = "hjku qaht kyog ulue"
destinatarios = ["rafael@newbasca.com.br", "victor@newbasca.com.br"]

# ✅ Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📁 Caminhos no Drive
base_path = "/content/drive/My Drive/RelatoriosEsquadrao"
pdfs_path = os.path.join(base_path, "PDFs")
os.makedirs(pdfs_path, exist_ok=True)

# 📄 Conteúdo HTML (exemplo pronto)
html_content = """
<!DOCTYPE html>
<html lang="pt">
  <head>
    <meta charset="UTF-8" />
    <title>Relatório Semanal - Esquadrão de Vendas</title>
  </head>
  <body style="margin:0; padding:0; font-family:Verdana, Geneva, Tahoma, sans-serif; background-color:#f2f2f2;">
    <table align="center" width="100%" cellpadding="0" cellspacing="0" style="max-width:600px; background-color:#ffffff; margin:auto; border-radius:10px; overflow:hidden;">
      <tr>
        <td>
          <img src="https://i.ibb.co/m508tWNg/imagem-fundo-CDC.png" alt="Imagem do Relatório" width="100%" style="display:block; border:0;"/>
        </td>
      </tr>
      <tr>
        <td style="padding: 30px;">
          <h1 style="font-size: 24px; color: #001D00; margin-bottom: 20px;">Relatório Semanal - Esquadrão de Vendas</h1>
          <p style="font-size: 16px; color: #333333; margin-bottom: 20px;">Olá! O relatório da semana está disponível.</p>
          <p style="font-size: 16px; color: #333333; margin-bottom: 30px;">Você pode acessá-lo clicando no botão abaixo:</p>
          <a href="https://drive.google.com/drive/folders/1TcjcbfxsBxdctl_2Ev1es3dYBxkLWehn?usp=sharing" target="_blank" style="display: inline-block; padding: 14px 28px; background-color: #00F106; color: #000000; text-decoration: none; font-weight: bold; border-radius: 5px;">Acessar no Power BI</a>
        </td>
      </tr>
      <tr>
        <td style="padding: 30px; background-color: #e6ffe6; border-top: 1px solid #cccccc;">
          <h2 style="font-size: 18px; color: #001D00; margin-bottom: 15px;">Instruções para acesso</h2>
          <p style="font-size: 14px; color: #333333; margin-bottom: 10px;">Se estiver com problemas para acessar o relatório, verifique:</p>
          <ul style="font-size: 14px; color: #333333; padding-left: 20px; margin-bottom: 15px;">
            <li>Certifique-se de estar conectado à sua conta Google com permissão.</li>
            <li>Solicite acesso ao administrador, se necessário.</li>
            <li>Use navegador atualizado (Chrome, Firefox, Edge).</li>
          </ul>
          <p style="font-size: 14px; color: #333333;">Dúvidas? Fale com o administrador do sistema.</p>
        </td>
      </tr>
    </table>
  </body>
</html>
"""

# 💾 Salvar o HTML no Drive
with open(os.path.join(base_path, 'email_template.html'), 'w', encoding='utf-8') as f:
    f.write(html_content)

# 🔍 Buscar o PDF mais recente na pasta "PDFs"
arquivos_pdf = [f for f in os.listdir(pdfs_path) if f.lower().endswith('.pdf')]
if not arquivos_pdf:
    raise FileNotFoundError("⚠️ Nenhum arquivo PDF encontrado na pasta /PDFs do Drive.")

mais_recente = max(arquivos_pdf, key=lambda x: os.path.getmtime(os.path.join(pdfs_path, x)))
caminho_pdf = os.path.join(pdfs_path, mais_recente)

# 📎 Carregar o PDF
with open(caminho_pdf, 'rb') as f:
    pdf_data = f.read()

# ✉️ Criar o e-mail
msg = MIMEMultipart()
msg['From'] = email_remetente
msg['To'] = ", ".join(destinatarios)
msg['Subject'] = "Relatório Semanal - Esquadrão de Vendas"
msg.attach(MIMEText(html_content, "html"))

# Anexo PDF
anexo = MIMEApplication(pdf_data, _subtype="pdf")
anexo.add_header('Content-Disposition', 'attachment', filename=mais_recente)
msg.attach(anexo)

# 🚀 Enviar e-mail
try:
    servidor = smtplib.SMTP("smtp.gmail.com", 587)
    servidor.starttls()
    servidor.login(email_remetente, senha_app)
    servidor.sendmail(email_remetente, destinatarios, msg.as_string())
    servidor.quit()
    print(f"✅ E-mail enviado com sucesso com o arquivo: {mais_recente}")

    # 🔁 Salvar cópia do PDF com data
    data_atual = datetime.today().strftime('%Y-%m-%d')
    novo_nome = f"relatorio-{data_atual}.pdf"
    with open(os.path.join(base_path, novo_nome), 'wb') as f:
        f.write(pdf_data)

    # 📝 Registrar no log
    log_path = os.path.join(base_path, "log_envios.txt")
    with open(log_path, 'a', encoding='utf-8') as log:
        hora = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log.write(f"[{hora}] Enviado: {mais_recente} → como {novo_nome}\n")

except Exception as e:
    print("❌ Erro ao enviar e-mail:", e)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ E-mail enviado com sucesso com o arquivo: relatorio semana 2.pdf
